In [1]:
import lightgbm as lgb
import sys
from os import listdir
import os.path
from os.path import isdir, isfile, join
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def readCSVfromDirectory(dir):
    print(dir)
    filepaths = []
    for f in listdir(dir):
        path = join(dir, f)
        if isfile(path) and f.endswith(".csv"):
            filepaths.append(path)

    if len(filepaths) == 0:
        exit("Directory does not contain any csv files: %s" % dir)

    filepaths = np.sort(filepaths)
    print("Found %s files." % len(filepaths))

    return filepaths


In [58]:
#############################################
#MARK: - Read Files

SOURCE_DIR = "./data/"


SOURCE_DIR = os.path.abspath(SOURCE_DIR)
if not isdir(SOURCE_DIR):
    exit("That's not a directory!")
else:
    print(SOURCE_DIR)

elems = listdir(SOURCE_DIR)
print(elems)
directories = {}
for e in elems:
    path = join(SOURCE_DIR, e)
    
    if isdir(path):
        filepaths = readCSVfromDirectory(path)

        directories[e] = []
        for filepath in filepaths:
            # the files contain values in [V] and [A]
            # use a current Factor of 10e-3 if given in [mA]
            dataframe = pd.read_csv(filepath, header=1)
            # Felix ist schuld
            dataframe.columns = dataframe.columns.str.strip().str.replace('(', '').str.replace(')', '') 
            #create id field
            dataframe = dataframe.assign(id= lambda x: dataframe.index )
            #remove ms field
            dataframe = dataframe.drop(columns=['ms'])
            if 'mA' in dataframe:
                dataframe = dataframe.assign(A= lambda x: x.mA / 1000)
                dataframe = dataframe.drop(columns=['mA'])
                #print("Converted mA to A")
            directories[e].append(dataframe)
            

/Users/paul/DataScienceProjekt/data
['ohp_voll-single.psdata', 'laptop-single', 'ohp_voll-laptop', 'lampe-ohp_voll.psdata', 'ohp_halb-single', 'ohp_halb-single.psdata', 'ohp_voll-laptop-bosch-lampe.psdata', 'lampe-ohp_voll', 'ohp_voll-single', 'ohp_halb-laptop-lampe.psdata', 'ohp_halb-laptop.psdata', 'bosch-single', 'lampe-single', 'ohp_voll-laptop-bosch-lampe', 'ohp_halb-laptop', 'ohp_halb-laptop-lampe']
/Users/paul/DataScienceProjekt/data/laptop-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_halb-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-ohp_voll
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/bosch-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop-bosch-lampe
Found 256 files.
/Users/paul/DataScienceProjekt/data/

In [60]:
for (id, dfs) in directories.items():
    print(id, len(dfs))
    print(dfs[0].head(4))
    print(dfs[0].describe())
    print('\n\n')

laptop-single 256
          V         A  id
0 -8.224738  0.018067   0
1 -8.224738  0.036012   1
2 -3.753777  0.018067   2
3  0.732444  0.018067   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.888648     0.021008  1222.000000
std     222.962687     0.338291   705.955027
min    -321.848200    -1.293191     0.000000
25%    -178.472200    -0.017823   611.000000
50%      81.362350     0.000122  1222.000000
75%     251.609800     0.018067  1833.000000
max     314.340600     1.419172  2444.000000



ohp_voll-laptop 256
           V         A  id
0 -12.710960 -1.078524   0
1  -8.224738 -1.078524   1
2  -3.753777 -1.078524   2
3  -8.224738 -1.078524   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.879487     0.443960  1222.000000
std     221.681810     2.560982   705.955027
min    -317.362000    -5.035249     0.000000
25%    -174.001300    -1.887875   611.000000
50%      81.362350 

#### TODO

- Zeit diskretisieren 0ms = 0 und 20ms = 2445 (neue spalte -> id des datensatzes) done
- mA zu A umrechnen (falls vorhanden) done
- geht multilabel mit gradient boosting? jaein, vielleicht
- in X und Y aufteilen 
- attacke!


In [62]:
features = []

for label, data in directories.items():
    # pro gerät alle daten in einen dataframe
    res = pd.concat(data, sort=False)
    #zu der tabelle eine spalte mit label hinzufügen
    res = res.assign(labels= lambda x: label)
    
    features.append(res)

# dataframes zu einem kombinieren
features = pd.concat(features, sort=False)

X = features.drop(columns=['labels'])
Y = features['labels']

In [63]:
len(X)

6259200

In [64]:
X

,V,A,id
0,-8.224738,0.018067,0
1,-8.224738,0.036012,1
2,-3.753777,0.018067,2
3,0.732444,0.018067,3
4,0.732444,0.036012,4
5,0.732444,0.018067,5
6,9.689627,0.018067,6
7,9.689627,0.036012,7
8,9.689627,0.018067,8
9,9.689627,0.018067,9


In [67]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [73]:
train_data = lgb.Dataset(X_train, label=Y_train)

test_data = lgb.Dataset(X_test, label=Y_test)

In [78]:
param = {'num_leaves':31, 'num_trees':100, 'objective':'binary'}
param['metric'] = ['auc', 'binary_logloss']
num_round = 10
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])

/Users/paul/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ValueError: could not convert string to float: 'ohp_voll-single'